In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
import statistics
from statistics import median
import random

import csv
import math

from scipy import ndimage
from scipy.signal import convolve2d
from scipy.stats import beta

import os
from os.path import exists

import cv2

from PIL import Image
import rasterio
from rasterio.mask import mask

import geojson
import json

import colorsys
import math

In [ ]:
import pytesseract
#pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract.exe'

In [ ]:
from datetime import datetime

In [ ]:
import multiprocessing

In [ ]:
PROCESSES = 40

In [ ]:
split_multiprocessing = True # Always set to True 
for_each_loop_global = PROCESSES

In [ ]:
map_preprocessing = True  # Set to True if map preprocessing is needed

crop_legend = True # Always set to True
smoothing_map = False # Always set to False

In [ ]:
print_intermediate_image = True # create directory and output intermediate images 
# output for cropped map => 'intermediate6/cropped_map_mask'
# output for polygon extraction => 'intermediate7(2)'

In [ ]:
data_dir='eval_data_perfomer' # path to input maps

# Below is a Preprocessing Step for All Maps

In [ ]:
#data_dir='validation'
if not os.path.exists('intermediate6/'):
    os.makedirs('intermediate6/')

candidate_file_name_for_preprocessing = []
for file_name in os.listdir(data_dir):
    if '.json' in file_name:
        filename=file_name.replace('.json', '.tif')
        file_path=os.path.join(data_dir, filename)
        test_json=file_path.replace('.tif', '.json')

        any_counter = 0

        with open(test_json) as f:
            gj = json.load(f)
        for this_gj in gj['shapes']:
            #print(this_gj)
            names = this_gj['label']
            features = this_gj['points']
            
            if '_poly' not in names and '_pt' not in names and '_line' not in names:
                print(names)
            any_counter = any_counter+1
            break
        
        if any_counter > 0:
            candidate_file_name_for_preprocessing.append(file_name)
print(len(candidate_file_name_for_preprocessing))
print(candidate_file_name_for_preprocessing)

In [ ]:
if not os.path.exists('intermediate6/cropped_map'):
    os.makedirs('intermediate6/cropped_map')
if not os.path.exists('intermediate6/cropped_map_mask'):
    os.makedirs('intermediate6/cropped_map_mask')

for target_file_q in range(0, len(candidate_file_name_for_preprocessing)):
    # get the .tif files
    file_name = candidate_file_name_for_preprocessing[target_file_q]

    if '.json' in file_name:
        filename=file_name.replace('.json', '.tif')
        print('Working on map:', file_name)
        file_path=os.path.join(data_dir, filename)
        test_json=file_path.replace('.tif', '.json')
        
        img0 = cv2.imread(file_path)
        rgb0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
        gray0 = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)

        blank = np.zeros((img0.shape[0],img0.shape[1],img0.shape[2]),dtype=np.uint8)
        blank[0:img0.shape[0],0:img0.shape[1],0:img0.shape[2]] = 255

        corner_area = np.copy(rgb0[int(rgb0.shape[0]*2/100):int(rgb0.shape[0]*3/100), int(rgb0.shape[1]*30/100):int(rgb0.shape[1]*70/100)])
        rgb_trimmed = np.zeros((corner_area.shape[2], corner_area.shape[0], corner_area.shape[1]), dtype='uint8')
        for dimension in range(0, 3):
            rgb_trimmed[dimension] = np.copy(corner_area[:,:,dimension])

        lower_color = np.array([250,250,250]) ### This shall be the color you want to crop off
        upper_color = np.array([256,256,256]) ### This shall be the color you want to crop off

        res_box = cv2.inRange(rgb0, lower_color, upper_color)

        # Either use threshold (less accurate, but works for rotated cases) or contour to remove black border
        '''
        lower_color = np.array([0,0,0])
        upper_color = np.array([1,1,1])

        res_box_1 = cv2.inRange(rgb0, lower_color, upper_color)
        res_box = cv2.bitwise_or(res_box, res_box_1)
        '''
        _,thresh = cv2.threshold(gray0,1,255,cv2.THRESH_BINARY)
        contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)

        res_box_1 = np.ones((img0.shape[0],img0.shape[1]),dtype=np.uint8)*255
        res_box_1[y:y+h,x:x+w] = 0
        res_box = cv2.bitwise_or(res_box, res_box_1)
        # Either use threshold or contour to remove black border


        res_box[res_box < 255] = 0
        img_bw00 = 255 - res_box

        # remove moisy white pixels before buffer
        kernel_before_blur00 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        opening_before_blur00 = cv2.morphologyEx(img_bw00, cv2.MORPH_OPEN, kernel_before_blur00, iterations=1)

        # smooth the image
        blur_radius = 5.0
        threshold_blur = 0
        gaussian_buffer00 = ndimage.gaussian_filter(opening_before_blur00, blur_radius)
        gaussian_buffer00[gaussian_buffer00 > threshold_blur] = 255

        # find connected components
        labeled00, nr_objects00 = ndimage.label(gaussian_buffer00 > threshold_blur)

        label_area = np.bincount(labeled00.flat)[1:] # 1
        arg_sort = np.argsort(label_area)

        check_generated = False
        threshold = [0.4, 0.3, 0.2, 0.1, 0.05]
        max_variety = 0
        for relaxing_threshold in range(0, len(threshold)):
            for target_arg in range(arg_sort.shape[0]-1, arg_sort.shape[0]-6, -1): # Only check the top-5 largest regions
                #print(target_arg, arg_sort[target_arg])
                selected_index = arg_sort[target_arg]+1 # 1

                selected_map_for_examination = np.zeros((labeled00.shape[0],labeled00.shape[1],1),dtype=np.uint8)
                selected_map_for_examination[labeled00 == selected_index] = 255
                
                # need to check
                crop_rgb0 = cv2.bitwise_and(rgb0,rgb0, mask=selected_map_for_examination)
                uniques = np.unique(crop_rgb0)
                if uniques.shape[0] > max_variety:
                    max_variety = uniques.shape[0]
                
                if uniques.shape[0] > 255*threshold[relaxing_threshold] or (relaxing_threshold > 0 and uniques.shape[0] == max_variety):
                    check_generated = True
                    break
            if check_generated:
                break
        
        if crop_legend == True:
            legend_mask = np.ones((img0.shape[0], img0.shape[1]), dtype='uint8') *255
            with open(test_json) as f:
                gj = json.load(f)
            for this_gj in gj['shapes']:
                names = this_gj['label']
                features = this_gj['points']
                geoms = np.array(features)
                y_min = int(np.min(geoms, axis=0)[0])
                y_max = int(np.max(geoms, axis=0)[0])
                x_min = int(np.min(geoms, axis=0)[1])
                x_max = int(np.max(geoms, axis=0)[1])
                legend_mask[x_min:x_max, y_min:y_max] = 0
            selected_map_for_examination = cv2.bitwise_and(selected_map_for_examination, legend_mask)

        if check_generated == False:
            selected_map_for_examination = np.ones((img0.shape[0],img0.shape[1],1),dtype=np.uint8)*255
            crop_rgb2 = np.copy(rgb0)
        else:
            blank_mask = cv2.bitwise_and(blank, blank, mask=cv2.bitwise_not(selected_map_for_examination))
            crop_rgb2 = cv2.add(crop_rgb0, blank_mask)

        # remove gray header
        header_region = np.copy(rgb0)
        header_region = header_region[0:int(rgb0.shape[0]*0.06),:]
        header_included = selected_map_for_examination[0:int(rgb0.shape[0]*0.06),:]

        lower_color = np.array([210,210,210]) ### This shall be the color you want to crop off
        upper_color = np.array([250,250,250]) ### This shall be the color you want to crop off
        header_gray = cv2.inRange(header_region, lower_color, upper_color)

        if np.sum(header_gray)/np.sum(header_included) > 0.85:
            print('remove gray header')
            selected_map_for_examination[0:int(rgb0.shape[0]*0.06),:] = 0

        out_file_path0='intermediate6/cropped_map_mask/'+file_name.replace('.json', '')+'_expected_crop_region.tif'
        cv2.imwrite(out_file_path0, selected_map_for_examination)
        
        
        plt.imshow(crop_rgb2)
        plt.show()

        print_bgr = cv2.cvtColor(crop_rgb2, cv2.COLOR_RGB2BGR)
        out_file_path0='intermediate6/cropped_map/'+file_name.replace('.json', '')+'_crop.tif'
        cv2.imwrite(out_file_path0, print_bgr)

# Belows are for Polygon Extraction Only

### Specify which maps to extract (map with polygon features)

In [ ]:
candidate_file_name_for_polygon = []
poly_legend_counter = []
#poly_legend_counter_v2 = []
for file_name in os.listdir(data_dir):
    if '.json' in file_name:
        filename=file_name.replace('.json', '.tif')
        #print('Working on map:', file_name)
        file_path=os.path.join(data_dir, filename)
        test_json=file_path.replace('.tif', '.json')
        
        poly_counter = 0
        legend_counter = 0
        poly_name_list = []

        with open(test_json) as f:
            gj = json.load(f)
        for this_gj in gj['shapes']:
            #print(this_gj)
            names = this_gj['label']
            features = this_gj['points']
            
            if '_poly' not in names and '_pt' not in names and '_line' not in names:
                print(names)
            if '_poly' not in names:
                continue
            if names not in poly_name_list:
                poly_name_list.append(names)
            legend_counter = legend_counter + 1
            
        if legend_counter > 0:
            poly_counter = poly_counter + 1
            poly_legend_counter.append(len(poly_name_list))
            #poly_legend_counter.append(legend_counter)
            #poly_legend_counter_v2.append(len(poly_name_list))
        
        if poly_counter > 0:
            candidate_file_name_for_polygon.append(file_name)
print(len(candidate_file_name_for_polygon))
print(candidate_file_name_for_polygon)
print(poly_legend_counter)
#print(poly_legend_counter_v2)

### Preprocessing for the map

In [ ]:
#data_dir='validation'
if not os.path.exists('intermediate7/'):
    os.makedirs('intermediate7/')


if map_preprocessing == True:
    runningtime_start=datetime.now()
    
    
    
    def preprocessing(map_id):
        file_name = candidate_file_name_for_polygon[map_id]
        
        
        filename=file_name.replace('.json', '.tif')
        print('Working on map:', file_name)
        file_path=os.path.join(data_dir, filename)
        test_json=file_path.replace('.tif', '.json')
        
        img0 = cv2.imread(file_path)
        hsv0 = cv2.cvtColor(img0, cv2.COLOR_BGR2HSV)
        rgb0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
        gray0 = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)


        lower_black = np.array([0,0,0])
        upper_black = np.array([130,130,130]) #100

        blank = np.zeros((img0.shape[0],img0.shape[1],img0.shape[2]),dtype=np.uint8)
        blank[0:img0.shape[0],0:img0.shape[1],0:img0.shape[2]]=255

        # create a mask to only preserve current legend color in the basemap
        mask_box = cv2.inRange(rgb0, lower_black, upper_black)
        res_box = cv2.bitwise_and(blank,blank, mask=mask_box)

        # convert to grayscale 
        detected_gray0 = cv2.cvtColor(res_box, cv2.COLOR_BGR2GRAY)
        img_bw0 = cv2.threshold(detected_gray0, 60, 255, cv2.THRESH_BINARY)[1] # 127

        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_blackregion.png'
        cv2.imwrite(out_file_path0, img_bw0)



        lower_black1 = np.array([100,100,100])
        upper_black1 = np.array([180,180,180])

        # create a mask to only preserve current legend color in the basemap
        mask_box1 = cv2.inRange(rgb0, lower_black1, upper_black1)
        res_box1 = cv2.bitwise_and(blank,blank, mask=mask_box1)

        # convert to grayscale 
        detected_gray1 = cv2.cvtColor(res_box1, cv2.COLOR_BGR2GRAY)
        img_bw01 = cv2.threshold(detected_gray1, 60, 255, cv2.THRESH_BINARY)[1] # 127

        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_grayregion.png'
        cv2.imwrite(out_file_path0, img_bw01)



        blank = np.zeros((img0.shape[0],img0.shape[1],img0.shape[2]),dtype=np.uint8)
        blank[0:img0.shape[0],0:img0.shape[1],0:img0.shape[2]] = 255

        corner_area = np.copy(rgb0[int(rgb0.shape[0]*2/100):int(rgb0.shape[0]*3/100), int(rgb0.shape[1]*30/100):int(rgb0.shape[1]*70/100)])
        rgb_trimmed = np.zeros((corner_area.shape[2], corner_area.shape[0], corner_area.shape[1]), dtype='uint8')
        for dimension in range(0, 3):
            rgb_trimmed[dimension] = np.copy(corner_area[:,:,dimension])

        lower_color = np.array([250,250,250]) ### This shall be the color you want to crop off
        upper_color = np.array([256,256,256]) ### This shall be the color you want to crop off

        res_box = cv2.inRange(rgb0, lower_color, upper_color)

        # Either use threshold (less accurate, but works for rotated cases) or contour to remove black border
        '''
        lower_color = np.array([0,0,0])
        upper_color = np.array([1,1,1])

        res_box_1 = cv2.inRange(rgb0, lower_color, upper_color)
        res_box = cv2.bitwise_or(res_box, res_box_1)
        '''
        _,thresh = cv2.threshold(gray0,1,255,cv2.THRESH_BINARY)
        contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)

        res_box_1 = np.ones((img0.shape[0],img0.shape[1]),dtype=np.uint8)*255
        res_box_1[y:y+h,x:x+w] = 0
        res_box = cv2.bitwise_or(res_box, res_box_1)
        # Either use threshold or contour to remove black border


        res_box[res_box < 255] = 0
        img_bw00 = 255 - res_box

        # remove moisy white pixels before buffer
        kernel_before_blur00 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        opening_before_blur00 = cv2.morphologyEx(img_bw00, cv2.MORPH_OPEN, kernel_before_blur00, iterations=1)

        # smooth the image
        blur_radius = 5.0
        threshold_blur = 0
        gaussian_buffer00 = ndimage.gaussian_filter(opening_before_blur00, blur_radius)
        gaussian_buffer00[gaussian_buffer00 > threshold_blur] = 255

        # find connected components
        labeled00, nr_objects00 = ndimage.label(gaussian_buffer00 > threshold_blur)

        label_area = np.bincount(labeled00.flat)[1:] # 1
        arg_sort = np.argsort(label_area)

        check_generated = False
        threshold = [0.4, 0.3, 0.2, 0.1, 0.05]
        temp_max = 0
        for relaxing_threshold in range(0, len(threshold)):
            for target_arg in range(arg_sort.shape[0]-1, arg_sort.shape[0]-6, -1):
                #print(target_arg, arg_sort[target_arg])
                selected_index = arg_sort[target_arg]+1 # 1

                selected_map_for_examination = np.zeros((labeled00.shape[0],labeled00.shape[1],1),dtype=np.uint8)
                selected_map_for_examination[labeled00 == selected_index] = 255
                
                # need to check
                crop_rgb0 = cv2.bitwise_and(rgb0,rgb0, mask=selected_map_for_examination)
                uniques = np.unique(crop_rgb0)
                #print(uniques.shape[0])
                if uniques.shape[0] > temp_max:
                    temp_max = uniques.shape[0]
                
                if uniques.shape[0] > 255*threshold[relaxing_threshold] or (relaxing_threshold > 0 and uniques.shape[0] == temp_max):
                    check_generated = True
                    break
            if check_generated:
                break
        
        if crop_legend == True:
            legend_mask = np.ones((img0.shape[0], img0.shape[1]), dtype='uint8') *255
            with open(test_json) as f:
                gj = json.load(f)
            for this_gj in gj['shapes']:
                names = this_gj['label']
                features = this_gj['points']
                geoms = np.array(features)
                y_min = int(np.min(geoms, axis=0)[0])
                y_max = int(np.max(geoms, axis=0)[0])
                x_min = int(np.min(geoms, axis=0)[1])
                x_max = int(np.max(geoms, axis=0)[1])
                legend_mask[x_min:x_max, y_min:y_max] = 0
            selected_map_for_examination = cv2.bitwise_and(selected_map_for_examination, legend_mask)
        
        if check_generated == False:
            selected_map_for_examination = np.ones((img0.shape[0],img0.shape[1],1),dtype=np.uint8)*255

            crop_rgb2 = np.copy(rgb0)
        else:
            # remove gray header
            header_region = np.copy(rgb0)
            header_region = header_region[0:int(rgb0.shape[0]*0.06),:]
            header_included = selected_map_for_examination[0:int(rgb0.shape[0]*0.06),:]

            lower_color = np.array([210,210,210]) ### This shall be the color you want to crop off
            upper_color = np.array([250,250,250]) ### This shall be the color you want to crop off
            header_gray = cv2.inRange(header_region, lower_color, upper_color)

            if np.sum(header_included) > 0 and np.sum(header_gray)/np.sum(header_included) > 0.85:
                print('remove gray header')
                selected_map_for_examination[0:int(rgb0.shape[0]*0.06),:] = 0
            
            blank_mask = cv2.bitwise_and(blank, blank, mask=cv2.bitwise_not(selected_map_for_examination))
            crop_rgb2 = cv2.add(crop_rgb0, blank_mask)
        #plt.imshow(crop_rgb2)
        #plt.show()
        
        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_expected_crop_region.tif'
        cv2.imwrite(out_file_path0, selected_map_for_examination)

        print_bgr = cv2.cvtColor(crop_rgb2, cv2.COLOR_RGB2BGR)
        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_crop.tif'
        cv2.imwrite(out_file_path0, print_bgr)


        #img_bw0_arg = np.argwhere(img_bw0 == 255)
        #print(img_bw0_arg.shape)

        img1 = np.copy(img0)
        rgb1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
        #print(rgb1.shape)
        '''
        crop_rgb1 = rgb1
        crop_img_bw0 = img_bw0
        crop_img_bw01 = img_bw01
        '''
        #print(rgb1.shape)
        #print(selected_map_for_examination.shape)

        
        #crop_rgb1 = cv2.bitwise_and(rgb1,rgb1, mask=selected_map_for_examination)
        crop_rgb1 = np.copy(crop_rgb2)
        crop_img_bw0 = cv2.bitwise_and(img_bw0,img_bw0, mask=selected_map_for_examination)
        crop_img_bw01 = cv2.bitwise_and(img_bw01,img_bw01, mask=selected_map_for_examination)
        
        


        #print(crop_rgb1.shape)
        plt.imshow(crop_rgb1)

        print_bgr = cv2.cvtColor(crop_rgb1, cv2.COLOR_RGB2BGR)
        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_crop.png'
        cv2.imwrite(out_file_path0, print_bgr)
        #break

        img_bw0_arg = np.argwhere(crop_img_bw0 == 255)
        #print(img_bw0_arg.shape)

        
        #plt.imshow(crop_img_bw01)
        #plt.show()
        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_crop_grayregion.png'
        cv2.imwrite(out_file_path0, crop_img_bw01)

        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_crop_blackregion.png'
        cv2.imwrite(out_file_path0, crop_img_bw0)




        trans_hsv1 = cv2.cvtColor(crop_rgb1, cv2.COLOR_RGB2HSV)

        print_bgr = cv2.cvtColor(crop_rgb1, cv2.COLOR_RGB2BGR)
        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_remove_black.png'
        cv2.imwrite(out_file_path0, print_bgr)


        # without mean shift (for rgb)
        trans_rgb0 = cv2.cvtColor(trans_hsv1, cv2.COLOR_HSV2RGB)
        trans_hsv0 = cv2.cvtColor(trans_rgb0, cv2.COLOR_RGB2HSV)
        #crop_rgb1 = np.copy(trans_rgb0)
        #trans_hsv1 = np.copy(trans_hsv0)
        
        # mean shift (for rgb and hsv)
        spatialRadius = 10
        colorRadius = 25
        maxPyramidLevel = 2

        img = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_remove_black.png')
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img_rgb_part = cv2.bitwise_and(img_rgb, img_rgb, mask=selected_map_for_examination)
        

        result = cv2.pyrMeanShiftFiltering(img_rgb, spatialRadius, colorRadius, maxPyramidLevel)

        result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

        crop_rgb1 = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        trans_hsv1 = cv2.cvtColor(crop_rgb1, cv2.COLOR_RGB2HSV)

        out_file_path0='intermediate7/'+file_name.replace('.json', '')+'_remove_black_mean_shift.png'
        cv2.imwrite(out_file_path0, result)
        
        return selected_map_for_examination
    
    
    # multiprocessing_preprocessing
    with multiprocessing.Pool(int(PROCESSES/2)) as pool:
        multiprocessing_results = [pool.apply_async(preprocessing, (this_map,)) for this_map in range(0, len(candidate_file_name_for_polygon))]

        for this_map in multiprocessing_results:
            this_crop_map = this_map.get()
            plt.imshow(this_crop_map)
            plt.show()
    
    
    print('time check _v00:', datetime.now()-runningtime_start)

In [ ]:
def generate_mask(given_size):
    sliding_window_size = given_size
    initial_mask = np.zeros((sliding_window_size*2+3,sliding_window_size*2+3), dtype=int)
    center_j = sliding_window_size+1
    center_i = sliding_window_size+1
    initial_mask[center_j-1:center_j+2, center_i-1:center_i+2] = -1

    initial_mask[center_j-3][center_i-1] = 2
    initial_mask[center_j-3][center_i+1] = 2
    initial_mask[center_j-2][center_i] = 2

    initial_mask[center_j-1][center_i+3] = 4
    initial_mask[center_j+1][center_i+3] = 4
    initial_mask[center_j][center_i+2] = 4

    initial_mask[center_j+3][center_i-1] = 6
    initial_mask[center_j+3][center_i+1] = 6
    initial_mask[center_j+2][center_i] = 6

    initial_mask[center_j-1][center_i-3] = 8
    initial_mask[center_j+1][center_i-3] = 8
    initial_mask[center_j][center_i-2] = 8


    for j in range(0, sliding_window_size-1):
        for i in range(center_i-(sliding_window_size-2-j), center_i+(sliding_window_size-2-j)+1):
            initial_mask[j][i] = 2

    for j in range(sliding_window_size+3+1, sliding_window_size*2+3):
        for i in range(center_i-(j-(sliding_window_size+3+1)), center_i+(j-(sliding_window_size+3+1))+1):
            initial_mask[i][j] = 4

    for j in range(sliding_window_size+3+1, sliding_window_size*2+3):
        for i in range(center_i-(j-(sliding_window_size+3+1)), center_i+(j-(sliding_window_size+3+1))+1):
            initial_mask[j][i] = 6

    for j in range(0, sliding_window_size-1):
        for i in range(center_i-(sliding_window_size-2-j), center_i+(sliding_window_size-2-j)+1):
            initial_mask[i][j] = 8

    initial_mask_arg = np.argwhere(initial_mask == 0)
    for i, j in initial_mask_arg:
        if i<=sliding_window_size and j<=sliding_window_size:
            initial_mask[i][j] = 1
        elif i<=sliding_window_size and j>=sliding_window_size:
            initial_mask[i][j] = 3
        elif i>=sliding_window_size and j>sliding_window_size:
            initial_mask[i][j] = 5
        elif i>=sliding_window_size and j<=sliding_window_size:
            initial_mask[i][j] = 7
    #print(initial_mask)


    masking = []
    for direction in range(0, 8):
        masking.append(np.zeros((sliding_window_size*2+3,sliding_window_size*2+3), dtype=bool))
    masking = np.array(masking)

    for direction in range(0, 8):
        initial_mask_arg = np.argwhere(initial_mask == (direction+1))
        for i, j in initial_mask_arg:
            masking[direction][i][j] = True
    #print(masking.shape)

    return masking

def kernel_assign_white(img, i, j):
    img[max(i-1, 0)][max(j-1, 0)] = 255
    img[max(i-1, 0)][j] = 255
    img[max(i-1, 0)][min(j+1, 0)] = 255
    img[i][max(j-1, 0)] = 255
    img[i][j] = 255
    img[i][min(j+1, 0)] = 255
    img[min(i+1, 0)][max(j-1, 0)] = 255
    img[min(i+1, 0)][j] = 255
    img[min(i+1, 0)][min(j+1, 0)] = 255

def center_assign_white(img, i, j):
    img[i][j] = 255

### Attempt to polygon extraction

In [ ]:
#data_dir='validation'
if not os.path.exists('intermediate7(2)/'):
    os.makedirs('intermediate7(2)/')
for target_file_q in range(0, len(candidate_file_name_for_polygon), 1):
    file_name = candidate_file_name_for_polygon[target_file_q]
    
    running_time_v = []
    
    
    # get the .tif files
    if '.json' in file_name:
        runningtime_start=datetime.now()


        filename=file_name.replace('.json', '.tif')
        print('Working on map:', file_name)
        file_path=os.path.join(data_dir, filename)
        test_json=file_path.replace('.tif', '.json')
        file_name_json = test_json.replace('.json', '.json')
        
        #print(test_json)
        img000 = cv2.imread(file_path)
        #hsv0 = cv2.cvtColor(img0, cv2.COLOR_BGR2HSV)
        #rgb0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
        img_bound = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_expected_crop_region.tif')
        img_bound = cv2.cvtColor(img_bound, cv2.COLOR_BGR2GRAY)

        img_crop_gray = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_crop_grayregion.png')
        img_crop_black = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_crop_blackregion.png')
        img_crop_gray = cv2.cvtColor(img_crop_gray, cv2.COLOR_BGR2GRAY)
        img_crop_black = cv2.cvtColor(img_crop_black, cv2.COLOR_BGR2GRAY)

        img_rb = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_remove_black.png')
        img_ms = cv2.imread('intermediate7/'+file_name.replace('.json', '')+'_remove_black_mean_shift.png')


        with open(file_name_json) as f:
            gj = json.load(f)
        json_height = gj['imageHeight']
        json_width = gj['imageWidth']
        rescale_factor_0 = 1.0
        rescale_factor_1 = 1.0



        ## Non-white background
        non_white_background = False
        if np.sum(img_bound) / 255 >= (img_bound.shape[0]*img_bound.shape[1]) * 0.99 or np.unique(img_bound).shape[0] == 1:
            lower_white = np.array([250,250,250])
            upper_white = np.array([256,256,256])
            mask_white_img000 = cv2.inRange(img000, lower_white, upper_white)
            lower_white = np.array([0,0,0])
            upper_white = np.array([130,130,130])
            mask_white_img000_2 = cv2.inRange(img000, lower_white, upper_white)
            mask_white_img000 = cv2.bitwise_or(mask_white_img000, mask_white_img000_2)

            corner_avg_white = np.sum(mask_white_img000[int(mask_white_img000.shape[0]*98/100): int(mask_white_img000.shape[0]*99/100), int(mask_white_img000.shape[1]*98/100): int(mask_white_img000.shape[1]*99/100)])/255.0
            corner_area = (int(mask_white_img000.shape[0]*99/100) - int(mask_white_img000.shape[0]*98/100)) * (int(mask_white_img000.shape[1]*99/100) - int(mask_white_img000.shape[1]*98/100))

            if corner_avg_white / corner_area < 0.66:
                non_white_background = True
                print('non_white_background')



        

        ### Legend is always not considered
        if True:
            for this_gj in gj['shapes']:
                #print(this_gj)
                names = this_gj['label']
                features = this_gj['points']

                geoms = np.array(features)
                y_min = int(np.min(geoms, axis=0)[0]*rescale_factor_1)
                y_max = int(np.max(geoms, axis=0)[0]*rescale_factor_1)
                x_min = int(np.min(geoms, axis=0)[1]*rescale_factor_0)
                x_max = int(np.max(geoms, axis=0)[1]*rescale_factor_0)

                legend_mask = np.ones((img_rb.shape[0], img_rb.shape[1]), dtype='uint8') *255
                legend_mask[x_min:x_max, y_min:y_max] = 0
                img_bound = cv2.bitwise_and(img_bound, legend_mask)
            img_rb = cv2.bitwise_and(img_rb, img_rb, mask=img_bound)
            img_ms = cv2.bitwise_and(img_ms, img_ms, mask=img_bound)
            img_crop_gray = cv2.bitwise_and(img_crop_gray, img_crop_gray, mask=img_bound)
            img_crop_black = cv2.bitwise_and(img_crop_black, img_crop_black, mask=img_bound)
        hsv_rb = cv2.cvtColor(img_rb, cv2.COLOR_BGR2HSV)
        rgb_rb = cv2.cvtColor(img_rb, cv2.COLOR_BGR2RGB)
        hsv_ms = cv2.cvtColor(img_ms, cv2.COLOR_BGR2HSV)
        rgb_ms = cv2.cvtColor(img_ms, cv2.COLOR_BGR2RGB)



        
        poly_counter = 0
        color_space = []
        color_avg = []
        map_name = file_name.replace('.json', '')
        legend_name = []
        legend_name_check = []
        extracted_legend_name = []


        hsv_space = np.zeros((255), dtype='uint8') # only for h space
        rgb_space = np.zeros((255,255,3), dtype='uint8')


        if not os.path.exists('intermediate7(2)/'+map_name):
            os.makedirs('intermediate7(2)/'+map_name)


        for this_gj in gj['shapes']:
            #if '_poly' not in names:
                #continue
            #print(this_gj)
            names = this_gj['label']
            features = this_gj['points']
            
            if '_poly' not in names:
                continue
            if names in legend_name_check:
                continue
            legend_name_check.append(names)
            legend_name.append(names.replace('_poly',''))

            poly_counter = poly_counter+1


            ### There is no groundtruth for validation data
            #print('training/'+map_name+'_'+names+'.tif')


            ### Read json source for the legend
            geoms = np.array(features)
            y_min = int(np.min(geoms, axis=0)[0]*rescale_factor_1)
            y_max = int(np.max(geoms, axis=0)[0]*rescale_factor_1)
            x_min = int(np.min(geoms, axis=0)[1]*rescale_factor_0)
            x_max = int(np.max(geoms, axis=0)[1]*rescale_factor_0)

            img_legend = np.zeros((x_max-x_min, y_max-y_min, 3), dtype='uint8')
            img_legend = np.copy(img000[x_min:x_max, y_min:y_max, :])
            
            if print_intermediate_image == True:
                out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+names+'_legend.tif'
                cv2.imwrite(out_file_path0, img_legend)
            
            
            img_legend = cv2.cvtColor(img_legend, cv2.COLOR_BGR2RGB)
            img_legend = img_legend[int(img_legend.shape[0]/8):int(img_legend.shape[0]*7/8), int(img_legend.shape[1]/8):int(img_legend.shape[1]*7/8), :]
            hsv_legend = cv2.cvtColor(img_legend, cv2.COLOR_RGB2HSV)
            black_threshold = 30 #130
            white_threshold = 250 #245

            lower_black_rgb_trimmed0 = np.array([0,0,0])
            upper_black_rgb_trimmed0 = np.array([130,130,130])
            mask_test_img_legend = cv2.inRange(img_legend, lower_black_rgb_trimmed0, upper_black_rgb_trimmed0)
            if np.sum(mask_test_img_legend == 255) > np.sum(img_legend > 0) * 0.25:
                black_threshold = 30
            
            rgb_trimmed = np.zeros((img_legend.shape[2], img_legend.shape[0], img_legend.shape[1]), dtype='uint8')
            hsv_trimmed = np.zeros((img_legend.shape[2], img_legend.shape[0], img_legend.shape[1]), dtype='uint8')
            rgb_trimmed = rgb_trimmed.astype(float)
            hsv_trimmed = hsv_trimmed.astype(float)
            for dimension in range(0, 3):
                rgb_trimmed[dimension] = np.copy(img_legend[:,:,dimension]).astype(float)
                hsv_trimmed[dimension] = np.copy(hsv_legend[:,:,dimension]).astype(float)

            rgb_trimmed_temp = np.copy(rgb_trimmed)
            rgb_trimmed[0, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan
            hsv_trimmed[0, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan
            rgb_trimmed[1, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan
            hsv_trimmed[1, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan
            rgb_trimmed[2, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan
            hsv_trimmed[2, np.logical_and.reduce([rgb_trimmed_temp[0]<=black_threshold, rgb_trimmed_temp[1]<=black_threshold, rgb_trimmed_temp[2]<=black_threshold])] = np.nan

            rgb_trimmed[0, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan
            hsv_trimmed[0, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan
            rgb_trimmed[1, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan
            hsv_trimmed[1, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan
            rgb_trimmed[2, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan
            hsv_trimmed[2, np.logical_and.reduce([rgb_trimmed_temp[0]>=white_threshold, rgb_trimmed_temp[1]>=white_threshold, rgb_trimmed_temp[2]>=white_threshold])] = np.nan



            if np.sum(np.isnan(hsv_trimmed)) >= (hsv_trimmed.shape[0]*hsv_trimmed.shape[1]*hsv_trimmed.shape[2]):
                color_space_holder = []
                rgb_lower_box = np.array((0,0,0), dtype='uint8')
                rgb_upper_box = np.array((0,0,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array((245,245,245), dtype='uint8')
                rgb_upper_box = np.array((255,255,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array((245,245,245), dtype='uint8')
                rgb_upper_box = np.array((255,255,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array((245,245,245), dtype='uint8')
                rgb_upper_box = np.array((255,255,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array((245,245,245), dtype='uint8')
                rgb_upper_box = np.array((255,255,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array((245,245,245), dtype='uint8')
                rgb_upper_box = np.array((255,255,255), dtype='uint8')
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)

                color_avg_holder = np.array((0,0,0), dtype='uint8')
            else:
                color_space_holder = []
                hsv_lower_box = np.array([int(np.nanquantile(hsv_trimmed[0],.2)),int(np.nanquantile(hsv_trimmed[1],.1)),int(np.nanquantile(hsv_trimmed[2],.1))]) #.2
                hsv_upper_box = np.array([int(np.nanquantile(hsv_trimmed[0],.8)),int(np.nanquantile(hsv_trimmed[1],.9)),int(np.nanquantile(hsv_trimmed[2],.9))]) #.8
                color_space_holder.append(hsv_lower_box)
                color_space_holder.append(hsv_upper_box)
                hsv_space[int(np.nanquantile(hsv_trimmed[0],.2)): int(np.nanquantile(hsv_trimmed[0],.8))] += poly_counter
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.2)),int(np.nanquantile(rgb_trimmed[1],.2)),int(np.nanquantile(rgb_trimmed[2],.2))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.8)),int(np.nanquantile(rgb_trimmed[1],.8)),int(np.nanquantile(rgb_trimmed[2],.8))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_space[int(np.nanquantile(rgb_trimmed[0],.3)): int(np.nanquantile(rgb_trimmed[0],.7)), int(np.nanquantile(rgb_trimmed[1],.3)): int(np.nanquantile(rgb_trimmed[1],.7)), int(np.nanquantile(rgb_trimmed[2],.3)): int(np.nanquantile(rgb_trimmed[2],.7))] = poly_counter
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.15)),int(np.nanquantile(rgb_trimmed[1],.15)),int(np.nanquantile(rgb_trimmed[2],.15))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.85)),int(np.nanquantile(rgb_trimmed[1],.85)),int(np.nanquantile(rgb_trimmed[2],.85))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.1)),int(np.nanquantile(rgb_trimmed[1],.1)),int(np.nanquantile(rgb_trimmed[2],.1))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.9)),int(np.nanquantile(rgb_trimmed[1],.9)),int(np.nanquantile(rgb_trimmed[2],.9))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.06)),int(np.nanquantile(rgb_trimmed[1],.06)),int(np.nanquantile(rgb_trimmed[2],.06))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.94)),int(np.nanquantile(rgb_trimmed[1],.94)),int(np.nanquantile(rgb_trimmed[2],.94))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.02)),int(np.nanquantile(rgb_trimmed[1],.02)),int(np.nanquantile(rgb_trimmed[2],.02))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.98)),int(np.nanquantile(rgb_trimmed[1],.98)),int(np.nanquantile(rgb_trimmed[2],.98))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)
                rgb_lower_box = np.array([int(np.nanquantile(rgb_trimmed[0],.01)),int(np.nanquantile(rgb_trimmed[1],.01)),int(np.nanquantile(rgb_trimmed[2],.01))])
                rgb_upper_box = np.array([int(np.nanquantile(rgb_trimmed[0],.99)),int(np.nanquantile(rgb_trimmed[1],.99)),int(np.nanquantile(rgb_trimmed[2],.99))])
                color_space_holder.append(rgb_lower_box)
                color_space_holder.append(rgb_upper_box)

                color_avg_holder = np.array([int(np.nanquantile(rgb_trimmed[0],.5)),int(np.nanquantile(rgb_trimmed[1],.5)),int(np.nanquantile(rgb_trimmed[2],.5))])

            color_space.append(color_space_holder)
            color_avg.append(color_avg_holder)

        print('time check _v0:', datetime.now()-runningtime_start)
        running_time_v.append(datetime.now()-runningtime_start)

        ans_category = np.zeros((poly_counter+1, img_rb.shape[0], img_rb.shape[1]), dtype='uint8')

        blank = np.ones((img_rb.shape[0],img_rb.shape[1]),dtype=np.uint8)*255
        ans_category[poly_counter] = np.zeros((img_rb.shape[0],img_rb.shape[1]),dtype=np.uint8)

        #print(legend_name)
        #print(color_space)




        # If there is color shift for all legends, but this section is never used for the final solution. (smoothing_map always set to False)
        if smoothing_map == True:
            color_avg.append(np.array([0,0,0]))
            #color_avg.append(np.array([255,255,255]))

            img_bound_argwhere = np.argwhere(img_bound)
            (ystart, xstart), (ystop, xstop) = img_bound_argwhere.min(0), img_bound_argwhere.max(0) + 1 
            im = np.copy(rgb_rb[ystart:ystop, xstart:xstop, :])
            image = im.reshape(im.shape[0],im.shape[1],1,3)

            # Create color container 
            colors_container = np.ones(shape=[image.shape[0],image.shape[1],len(color_avg),3])
            for i,color in enumerate(color_avg):
                colors_container[:,:,i,:] = color
            
            rgb_weight = np.ones(shape=[image.shape[0],image.shape[1],1,3])
            rgb_weight[:,:,:,0] = 1 # 2
            rgb_weight[:,:,:,1] = 1 # 4
            rgb_weight[:,:,:,2] = 1 # 3

            background_correction_direct_rgb = np.ones(shape=[image.shape[0],image.shape[1],1,3])
            background_correction_direct_rgb[:,:,:,0] = 0.9
            background_correction_direct_rgb[:,:,:,1] = 0.9
            background_correction_direct_rgb[:,:,:,2] = 0.9

            image_deviation = np.zeros(shape=[image.shape[0],image.shape[1],1,3])
            image_deviation[:,:,:,0] = image[:,:,:,0] - image[:,:,:,1]
            image_deviation[:,:,:,1] = image[:,:,:,0] - image[:,:,:,2]
            image_deviation[:,:,:,2] = image[:,:,:,1] - image[:,:,:,2]

            legend_deviation = np.zeros(shape=[image.shape[0],image.shape[1],len(color_avg),3])
            legend_deviation[:,:,:,0] = colors_container[:,:,:,0] - colors_container[:,:,:,1]
            legend_deviation[:,:,:,1] = colors_container[:,:,:,0] - colors_container[:,:,:,2]
            legend_deviation[:,:,:,2] = colors_container[:,:,:,1] - colors_container[:,:,:,2]
            
            background_correction_deviated_rgb = np.ones(shape=[image.shape[0],image.shape[1],1,3])
            background_correction_deviated_rgb[:,:,:,:] = 0.5 + 0.5*(1.0-abs(image_deviation[:,:,:,:])/255.0)

            print('processing _v0 >>> _v1 (finding the closest color)... :', datetime.now()-runningtime_start)


            def closest(image,color_container):
                shape = image.shape[:2]
                total_shape = shape[0]*shape[1]

                # calculate distances
                distances_0 = np.sqrt(np.sum(rgb_weight*((color_container*background_correction_direct_rgb-image)**2),axis=3))
                distances_1 = np.sqrt(np.sum(((legend_deviation*background_correction_deviated_rgb-image_deviation)**2),axis=3))
                distances = distances_0*0.9 + distances_1*0.1

                min_index = np.argmin(distances,axis=2).reshape(-1)
                natural_index = np.arange(total_shape)

                reshaped_container = colors_container.reshape(-1,len(color_avg),3)

                color_view = reshaped_container[natural_index,min_index].reshape(shape[0],shape[1],3)
                return color_view, distances
            
            result_image, distances = closest(image,colors_container)
            result_image = result_image.astype(np.uint8)

            plt.imshow(result_image)
            plt.show()
            #Image.fromarray(result_image.astype(np.uint8)).show()




            subtract_rgb = []
            
            def extraction_step1(legend):
                extracted_map = cv2.inRange(result_image, color_avg[legend], color_avg[legend])

                distance_threshold = 0
                while (np.sum(extracted_map)/255 <= np.sum(img_bound)/255 *0.002):
                    distances_v = np.zeros((len(color_avg), image.shape[0], image.shape[1]), dtype='uint8')
                    distance_threshold = distance_threshold + 10

                    for legend in range(0, poly_counter):
                        distances_v[legend] = np.copy(distances[:,:,legend])
                        
                        extracted_map[distances_v[legend] <= distance_threshold] = 255
                        #extracted_map = cv2.bitwise_and(extracted_map, img_bound)
                        #ans_category[legend] = cv2.bitwise_or(ans_category[legend], extracted_map)

                        print(legend_name[legend])
                        plt.imshow(extracted_map)
                        plt.show()


                extracted_map_v = np.zeros((3, extracted_map.shape[0], extracted_map.shape[1]), dtype='uint8')
                extracted_map_v = extracted_map_v.astype(float)
                extracted_avg_rgb = []
                extracted_lower_rgb = []
                extracted_upper_rgb = []

                is_null = False
                for dimension in range(0, 3):
                    extracted_map_v[dimension] = np.copy(im[:,:,dimension]).astype(float)
                    extracted_map_v[dimension] = cv2.bitwise_and(im[:,:,dimension], im[:,:,dimension], mask=extracted_map)
                    extracted_map_v[dimension] = extracted_map_v[dimension]
                    extracted_map_v[dimension][extracted_map_v[dimension] == 0] = np.nan
                    if np.sum(np.isnan(extracted_map_v[dimension])) >= (im.shape[0] * im.shape[1]):
                        is_null = True
                        print('This situation shall not happen.')
                    else:
                        extracted_avg_rgb.append(int(np.nanquantile(extracted_map_v[dimension],.5)))
                        extracted_lower_rgb.append(int(np.nanquantile(extracted_map_v[dimension],.2)))
                        extracted_upper_rgb.append(int(np.nanquantile(extracted_map_v[dimension],.8)))
                #subtract_rgb.append(color_avg[legend] - extracted_avg_rgb)

                rgb_ms_masked = cv2.inRange(rgb_ms, np.array(extracted_lower_rgb), np.array(extracted_upper_rgb))
                rgb_rb_masked = cv2.inRange(rgb_rb, np.array(extracted_lower_rgb), np.array(extracted_upper_rgb))
                rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)
                
                img_masked = cv2.bitwise_or(rgb_masked, rgb_masked)
                
                if print_intermediate_image == True:
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v1_rgb.png'
                    cv2.imwrite(out_file_path0, rgb_masked)
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v2.png'
                    cv2.imwrite(out_file_path0, img_masked)
                
                return legend, img_masked, (color_avg[legend] - extracted_avg_rgb)
            
            
            # multiprocessing_step1
            with multiprocessing.Pool(PROCESSES) as pool:
                multiprocessing_results = [pool.apply_async(extraction_step1, (this_poly,)) for this_poly in range(0, poly_counter)]

                for this_poly in multiprocessing_results:
                    legend, img_masked, this_subtract_rgb = this_poly.get()
                    # add masked result into private ans_category
                    ans_category[legend] = np.copy(img_masked)
                    # add mophological result into global ans_category
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    img_masked_morphology = cv2.morphologyEx(img_masked, cv2.MORPH_OPEN, kernel, iterations=1)
                    img_masked_morphology[img_masked_morphology > 0] = legend+1
                    ans_category[poly_counter] = cv2.add(ans_category[poly_counter], img_masked_morphology)
                    
                    for space in range(2, len(color_space[legend]), 2):
                        color_space[legend][space] = color_space[legend][space] - this_subtract_rgb
                        color_space[legend][space+1] = color_space[legend][space+1] - this_subtract_rgb + 1
                    
            
            print('processing _v1 >>> _v2 (legend '+str(legend+1)+'/'+str(poly_counter)+')... :', datetime.now()-runningtime_start)


            result_image0 = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)
            out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_nearest.png'
            cv2.imwrite(out_file_path0, result_image0)



        else:
            
            
            
            def extraction_step2(legend):
                # create a mask to only preserve current legend color in the basemap
                hsv_ms_masked = cv2.inRange(hsv_ms, color_space[legend][0], color_space[legend][1])
                hsv_rb_masked = cv2.inRange(hsv_rb, color_space[legend][0], color_space[legend][1])
                hsv_masked = cv2.bitwise_or(hsv_ms_masked, hsv_rb_masked)

                rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][2], color_space[legend][3])
                rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][2], color_space[legend][3])
                rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)

                img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)


                if poly_counter <= 1:
                    # dilation if needed
                    color_space[legend][0][0] = color_space[legend][0][0] - 20
                    color_space[legend][1][0] = color_space[legend][1][0] + 20
                    color_space[legend][0][1] = color_space[legend][0][1] - 40
                    color_space[legend][1][1] = color_space[legend][1][1] + 40
                    color_space[legend][0][2] = color_space[legend][0][2] - 40
                    color_space[legend][1][2] = color_space[legend][1][2] + 40
                    #print(color_space[legend][0], color_space[legend][1])
                    
                    hsv_ms_masked = cv2.inRange(hsv_ms, color_space[legend][0], color_space[legend][1])
                    hsv_rb_masked = cv2.inRange(hsv_rb, color_space[legend][0], color_space[legend][1])
                    hsv_masked = cv2.bitwise_or(hsv_ms_masked, hsv_rb_masked)
                    
                    while poly_counter <= 1 and (np.sum(rgb_masked)/255 <= np.sum(img_bound)/255 *0.02):
                        # dilation if needed
                        for rgb_set in range(2, len(color_space[legend]), 2):
                            color_space[legend][rgb_set] = color_space[legend][rgb_set] -4
                            color_space[legend][rgb_set+1] = color_space[legend][rgb_set+1] +4
                        #print(color_space[legend][2], color_space[legend][3])

                        rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][2], color_space[legend][3])
                        rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][2], color_space[legend][3])
                        rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)

                        img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)
                    
                    # remove noisy white pixel
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (8,8))
                    opening = cv2.morphologyEx(img_masked, cv2.MORPH_OPEN, kernel, iterations=1)
                    img_masked=cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY)[1]
                        
                    if (np.sum(rgb_masked)/255 <= np.sum(img_bound)/255 *0.1):
                        for rgb_set in range(len(color_space[legend])-2, len(color_space[legend]), 2):
                            color_space[legend][rgb_set] = color_space[legend][rgb_set] -15
                            color_space[legend][rgb_set+1] = color_space[legend][rgb_set+1] +15
                            
                elif poly_counter <= 3:
                    # dilation if needed
                    color_space[legend][0][0] = color_space[legend][0][0] - 6
                    color_space[legend][1][0] = color_space[legend][1][0] + 6
                    color_space[legend][0][1] = color_space[legend][0][1] - 20
                    color_space[legend][1][1] = color_space[legend][1][1] + 20
                    color_space[legend][0][2] = color_space[legend][0][2] - 20
                    color_space[legend][1][2] = color_space[legend][1][2] + 20
                    #print(color_space[legend][0], color_space[legend][1])

                    hsv_ms_masked = cv2.inRange(hsv_ms, color_space[legend][0], color_space[legend][1])
                    hsv_rb_masked = cv2.inRange(hsv_rb, color_space[legend][0], color_space[legend][1])
                    hsv_masked = cv2.bitwise_or(hsv_ms_masked, hsv_rb_masked)

                    rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][2], color_space[legend][3])
                    rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][2], color_space[legend][3])
                    rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)

                    img_masked = cv2.bitwise_and(hsv_masked, hsv_masked)
                elif poly_counter > 3:
                    if poly_counter > 3:
                        # dilation if needed
                        current_h_lower = color_space[legend][0][0]
                        current_h_max = 0
                        for h_space in range(current_h_lower, -1, -1):
                            if hsv_space[h_space] > 0 and hsv_space[h_space] != legend:
                                current_h_max = h_space
                                break
                        
                        current_h_upper = color_space[legend][1][0]
                        current_h_min = 255
                        for h_space in range(current_h_upper, 255):
                            if hsv_space[h_space] > 0 and hsv_space[h_space] != legend:
                                current_h_min = h_space
                                break
                        
                        color_space[legend][0][0] = current_h_lower - min(int((current_h_lower-current_h_max)/4), 2)
                        color_space[legend][1][0] = current_h_upper + min(int((current_h_min-current_h_upper)/4), 2)
                        color_space[legend][0][1] = color_space[legend][0][1] - 5
                        color_space[legend][1][1] = color_space[legend][1][1] + 5
                        color_space[legend][0][2] = color_space[legend][0][2] - 5
                        color_space[legend][1][2] = color_space[legend][1][2] + 5
                        #print(color_space[legend][0], color_space[legend][1])
                        
                        hsv_ms_masked = cv2.inRange(hsv_ms, color_space[legend][0], color_space[legend][1])
                        hsv_rb_masked = cv2.inRange(hsv_rb, color_space[legend][0], color_space[legend][1])
                        hsv_masked = cv2.bitwise_or(hsv_ms_masked, hsv_rb_masked)

                        img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)
                    
                    hsv_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    hsv_opening = cv2.morphologyEx(hsv_masked, cv2.MORPH_OPEN, hsv_kernel, iterations=1)
                    dilation_step = 0
                    dilation_step_threshold = 10
                    while ((dilation_step<dilation_step_threshold) and (np.sum(hsv_opening)/255 <= np.sum(img_bound)/255 *0.0005)) or ((dilation_step>=dilation_step_threshold) and (np.sum(hsv_masked)/255 <= np.sum(img_bound)/255 *0.0005)):
                        # dilation if needed
                        current_h_lower = color_space[legend][0][0]
                        current_h_max = 0
                        for h_space in range(current_h_lower, -1, -1):
                            if hsv_space[h_space] > 0 and hsv_space[h_space] != legend:
                                current_h_max = h_space
                                break
                        
                        current_h_upper = color_space[legend][1][0]
                        current_h_min = 255
                        for h_space in range(current_h_upper, 255):
                            if hsv_space[h_space] > 0 and hsv_space[h_space] != legend:
                                current_h_min = h_space
                                break
                        
                        color_space[legend][0][0] = current_h_lower - min(int((current_h_lower-current_h_max)/4), 2)
                        color_space[legend][1][0] = current_h_upper + min(int((current_h_min-current_h_upper)/4), 2)
                        color_space[legend][0][1] = color_space[legend][0][1] - 5
                        color_space[legend][1][1] = color_space[legend][1][1] + 5
                        color_space[legend][0][2] = color_space[legend][0][2] - 5
                        color_space[legend][1][2] = color_space[legend][1][2] + 5
                        #print(color_space[legend][0], color_space[legend][1])

                        hsv_ms_masked = cv2.inRange(hsv_ms, color_space[legend][0], color_space[legend][1])
                        hsv_rb_masked = cv2.inRange(hsv_rb, color_space[legend][0], color_space[legend][1])
                        hsv_masked = cv2.bitwise_or(hsv_ms_masked, hsv_rb_masked)

                        img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)
                        
                        dilation_step = dilation_step + 1
                        
                        hsv_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        hsv_opening = cv2.morphologyEx(hsv_masked, cv2.MORPH_OPEN, hsv_kernel, iterations=1)
                        
                        if (dilation_step >= dilation_step_threshold*3):
                            break
                            
                    if poly_counter < 20 and  (dilation_step >= dilation_step_threshold*3) and (np.sum(hsv_masked)/255 <= np.sum(img_bound)/255 *0.0005):
                        # tends to become darker
                        for rgb_set in range(2, len(color_space[legend]), 2):
                            color_space[legend][rgb_set] = color_space[legend][rgb_set] -15
                            color_space[legend][rgb_set+1] = color_space[legend][rgb_set+1] +5
                        #print(color_space[legend][2], color_space[legend][3])
                        
                        rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][2], color_space[legend][3])
                        rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][2], color_space[legend][3])
                        rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)

                        img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)

                    while poly_counter > 3 and (np.sum(rgb_masked)/255 <= np.sum(img_bound)/255 *0.001):
                        # dilation if needed
                        for rgb_set in range(2, len(color_space[legend]), 2):
                            color_space[legend][rgb_set] = color_space[legend][rgb_set] -1
                            color_space[legend][rgb_set+1] = color_space[legend][rgb_set+1] +1
                        #print(color_space[legend][2], color_space[legend][3])

                        rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][2], color_space[legend][3])
                        rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][2], color_space[legend][3])
                        rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)

                        img_masked = cv2.bitwise_or(hsv_masked, rgb_masked)

                    if (np.sum(rgb_masked)/255 <= np.sum(img_bound)/255 *0.1):
                        for rgb_set in range(len(color_space[legend])-2, len(color_space[legend]), 2):
                            color_space[legend][rgb_set] = color_space[legend][rgb_set] -15
                            color_space[legend][rgb_set+1] = color_space[legend][rgb_set+1] +15
                    
                    if non_white_background == True:
                        if (np.sum(rgb_masked)/255 <= np.sum(img_bound)/255 *0.8):
                            img_masked = cv2.bitwise_and(hsv_masked, hsv_masked)
                            
                if print_intermediate_image == True:
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v1_hsv.png'
                    cv2.imwrite(out_file_path0, hsv_masked)
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v1_rgb.png'
                    cv2.imwrite(out_file_path0, rgb_masked)
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v2.png'
                    cv2.imwrite(out_file_path0, img_masked)
                
                return legend, img_masked, color_space[legend]
            
            
            # multiprocessing_step2
            with multiprocessing.Pool(PROCESSES) as pool:
                multiprocessing_results = [pool.apply_async(extraction_step2, (this_poly,)) for this_poly in range(0, poly_counter)]

                for this_poly in multiprocessing_results:
                    legend, img_masked, this_updated_color_space = this_poly.get()
                    # add masked result into private ans_category
                    ans_category[legend] = np.copy(img_masked)
                    # add mophological result into global ans_category
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    img_masked_morphology = cv2.morphologyEx(img_masked, cv2.MORPH_OPEN, kernel, iterations=1)
                    img_masked_morphology[img_masked_morphology > 0] = legend+1
                    ans_category[poly_counter] = cv2.add(ans_category[poly_counter], img_masked_morphology)
                    
                    color_space[legend] = np.copy(this_updated_color_space)
            
            
            
            print('processing _v0 >>> _v2 (legend '+str(legend+1)+'/'+str(poly_counter)+')... :', datetime.now()-runningtime_start)
        print('time check _v2:', datetime.now()-runningtime_start)
        running_time_v.append(datetime.now()-runningtime_start)

        
        
        if split_multiprocessing == True and poly_counter > 150: # split legends into multiple parts, multiprocessing for part of legends at a time => recommended if 'more than [around 150] legends in a map'
            for_each_loop = for_each_loop_global
            looping_times = math.ceil(poly_counter/for_each_loop)
        else: # = direct multiprocessing for all legends at a time => recommended if runnable
            looping_times = 1
            for_each_loop = poly_counter
        
        for looping in range(0, looping_times):
            range_min = 0 + for_each_loop*looping
            range_max = min(for_each_loop + for_each_loop*looping, poly_counter)
            print('looping... (round: '+str(looping+1)+'/'+str(looping_times)+')... (legend: '+str(range_min)+'-'+str(range_max)+' /'+str(poly_counter)+')')
            

            for iteration_relaxing in range(0, 4):
                global_solution = np.copy(ans_category[poly_counter])
                global_solution[global_solution > 0] = 0
                global_solution_empty = 255 - global_solution
                global_solution_empty = cv2.bitwise_and(global_solution_empty, img_bound)
                ans_category[poly_counter] = np.zeros((img_rb.shape[0],img_rb.shape[1]),dtype=np.uint8)

                # multiprocessing_step3
                def extraction_step3(legend):
                    # fetch current result for this legend
                    this_current_result = np.copy(ans_category[legend])

                    # create a mask to only preserve current legend color in the basemap
                    rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][4 + iteration_relaxing*2], color_space[legend][5 + iteration_relaxing*2])
                    rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][4 + iteration_relaxing*2], color_space[legend][5 + iteration_relaxing*2])
                    rgb_masked = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)
                    rgb_masked = rgb_masked.astype('uint8')

                    # remove moisy white pixels before buffer
                    kernel_before_blur = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    opening_before_blur = cv2.morphologyEx(this_current_result, cv2.MORPH_OPEN, kernel_before_blur, iterations=1)

                    # smooth the image
                    blur_radius = 1.0
                    threshold_blur = 0
                    gaussian_buffer = ndimage.gaussian_filter(opening_before_blur, blur_radius)
                    gaussian_buffer[gaussian_buffer > threshold_blur] = 255

                    current_empty = 255 - this_current_result
                    current_relaxing = cv2.bitwise_and(current_empty, gaussian_buffer)

                    relaxing_mask = cv2.bitwise_or(rgb_masked, img_crop_black)
                    relaxing_mask = cv2.bitwise_or(relaxing_mask, img_crop_gray)

                    relaxing_mask = cv2.bitwise_and(relaxing_mask, global_solution_empty)
                    relaxing_mask = relaxing_mask.astype('uint8')
                    temp_relax = cv2.bitwise_and(current_relaxing, current_relaxing, mask=relaxing_mask)

                    this_next_result = cv2.bitwise_or(this_current_result, temp_relax)

                    if iteration_relaxing == 3:
                        if print_intermediate_image == True:
                            out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v3.png'
                            cv2.imwrite(out_file_path0, ans_category[legend])
                    #print('v3_ ', iteration_relaxing, legend)

                    return legend, this_next_result

                # multiprocessing_step3
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(extraction_step3, (this_poly,)) for this_poly in range(range_min, range_max)]

                    for this_poly in multiprocessing_results:
                        legend, this_next_result = this_poly.get()
                        # add masked result into private ans_category
                        ans_category[legend] = np.copy(this_next_result)
                        # add mophological result into global ans_category
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        img_masked_morphology = cv2.morphologyEx(this_next_result, cv2.MORPH_OPEN, kernel, iterations=1)
                        img_masked_morphology[img_masked_morphology > 0] = legend+1
                        ans_category[poly_counter] = cv2.add(ans_category[poly_counter], img_masked_morphology)

                
                print('processing _v2 >>> _v3 (iteration '+str(iteration_relaxing+1)+'/4)... (legend '+str(legend+1)+'/'+str(poly_counter)+')... :', datetime.now()-runningtime_start)
            #for legend in range(0, poly_counter):
                #out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v3.png'
                #cv2.imwrite(out_file_path0, ans_category[legend])

            print('time check _v3:', datetime.now()-runningtime_start)
            running_time_v.append(datetime.now()-runningtime_start)



            img_crop_black_and_gray = cv2.bitwise_or(img_crop_black, img_crop_gray)





            # keep record of updated region
            #updated_region = np.zeros(poly_counter)
            updated_region = []
            updated_for_relaxing = np.ones((ans_category[poly_counter].shape[0],ans_category[poly_counter].shape[1]),dtype=np.uint8)*255

            # fill ip white pixel (remove noisy black pixel)
            for iteration in range(0, 1):
                global_solution = np.copy(ans_category[poly_counter])
                global_solution_temp = np.copy(ans_category[poly_counter])
                global_solution_temp[global_solution_temp > 0] = 0
                global_solution_empty = 255 - global_solution_temp
                global_solution_empty = cv2.bitwise_and(global_solution_empty, img_bound)
                ans_category[poly_counter] = np.zeros((img_rb.shape[0],img_rb.shape[1]),dtype=np.uint8)

                #updated_region = []
                next_updated_region = []


                def extraction_step4(legend):
                    # fetch current result for this legend
                    this_current_result = np.copy(ans_category[legend])

                    if iteration == 0:
                        updated_for_relaxing = np.ones((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)*255
                        # remove moisy white pixels before buffer
                        kernel_before_blur = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        opening_before_blur = cv2.morphologyEx(this_current_result, cv2.MORPH_OPEN, kernel_before_blur, iterations=1)
                    else:
                        #updated_for_relaxing = np.copy(updated_region[legend])
                        #opening_before_blur = np.copy(updated_for_relaxing)
                        updated_for_relaxing = np.ones((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)*255
                        # remove moisy white pixels before buffer
                        kernel_before_blur = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        opening_before_blur = cv2.morphologyEx(this_current_result, cv2.MORPH_OPEN, kernel_before_blur, iterations=1)

                        #print(np.sum(updated_for_relaxing))
                        if np.sum(updated_for_relaxing) == 0:
                            return legend, this_current_result, updated_for_relaxing
                        masked_update = cv2.bitwise_and(rgb_rb, rgb_rb, mask=updated_for_relaxing)

                        color_dynamic_placeholder_0 = []
                        color_dynamic_placeholder_1 = []
                        for dimension in range(0, 3):
                            masked_update_np = np.copy(masked_update[:,:,dimension]).astype(float)
                            masked_update_np[masked_update_np==0] = np.nan

                            color_dynamic_placeholder_0.append(int(np.nanquantile(masked_update_np,.25)))
                            color_dynamic_placeholder_1.append(int(np.nanquantile(masked_update_np,.75)))


                    # smooth the image
                    blur_radius = 5.0 # 5.0
                    threshold_blur = 255*0.33 # *0.33
                    gaussian_buffer = ndimage.gaussian_filter(opening_before_blur, blur_radius)
                    gaussian_buffer[gaussian_buffer > threshold_blur] = 255
                    gaussian_buffer[gaussian_buffer <= threshold_blur] = 0

                    current_empty = 255 - this_current_result
                    current_relaxing = cv2.bitwise_and(current_empty, gaussian_buffer)
                    #current_relaxing = cv2.bitwise_and(current_relaxing, updated_for_relaxing)  # shall not bitwise_and here <= already considered from gaussian_buffer <= opening_before_blur

                    current_relaxing = cv2. bitwise_and(current_relaxing, global_solution_empty)
                    #current_relaxing_arg = np.argwhere(current_relaxing == 255)

                    updated_for_relaxing = np.zeros((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)


                    if iteration == 0:
                        rgb_ms_masked = cv2.inRange(rgb_ms, color_space[legend][4 + 3*2], color_space[legend][5 + 3*2])
                        rgb_rb_masked = cv2.inRange(rgb_rb, color_space[legend][4 + 3*2], color_space[legend][5 + 3*2])

                        rgb_masked_dynamic = cv2.bitwise_or(rgb_ms_masked, rgb_rb_masked)
                        rgb_masked_dynamic = rgb_masked_dynamic.astype('uint8')
                    elif iteration >= 1 and iteration <= 1:
                    #else:
                        rgb_dynamic_lower_box = np.array([color_dynamic_placeholder_0[0], color_dynamic_placeholder_0[1], color_dynamic_placeholder_0[2]])
                        rgb_dynamic_upper_box = np.array([color_dynamic_placeholder_1[0], color_dynamic_placeholder_1[1], color_dynamic_placeholder_1[2]])

                        # create a mask to only preserve current legend color in the basemap
                        rgb_masked_dynamic = cv2.inRange(rgb_rb, rgb_dynamic_lower_box, rgb_dynamic_upper_box)
                        rgb_masked_dynamic = rgb_masked_dynamic.astype('uint8')


                    masking_targeted_color = cv2.bitwise_and(current_relaxing, rgb_masked_dynamic)
                    masking_black_and_gray = cv2.bitwise_and(current_relaxing, img_crop_black_and_gray)
                    #masking_self = cv2.bitwise_and(current_relaxing, this_current_result)

                    if iteration == 0:
                        this_next_result = cv2.bitwise_or(this_current_result, masking_targeted_color)
                        this_next_result = cv2.bitwise_or(this_next_result, masking_black_and_gray)
                    else:
                        this_next_result = cv2.bitwise_or(this_current_result, masking_targeted_color)
                        this_next_result = cv2.bitwise_or(this_next_result, masking_black_and_gray)

                    updated_for_relaxing = cv2.subtract(this_next_result, this_current_result)
                    
                    if print_intermediate_image == True:
                        out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v4.png'
                        cv2.imwrite(out_file_path0, this_next_result)

                    return legend, this_next_result, updated_for_relaxing


                # multiprocessing_step4
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(extraction_step4, (this_poly,)) for this_poly in range(range_min, range_max)]

                    for this_poly in multiprocessing_results:
                        legend, this_next_result, updated_for_relaxing = this_poly.get()
                        # add masked result into private ans_category
                        ans_category[legend] = np.copy(this_next_result)
                        # add mophological result into global ans_category
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        img_masked_morphology = cv2.morphologyEx(this_next_result, cv2.MORPH_OPEN, kernel, iterations=1)
                        img_masked_morphology[img_masked_morphology > 0] = legend+1
                        ans_category[poly_counter] = cv2.add(ans_category[poly_counter], img_masked_morphology)

                        next_updated_region.append(np.copy(updated_for_relaxing))



                

                updated_region = np.array(np.copy(next_updated_region))
                print('processing _v3 >>> _v4 (iteration '+str(iteration+1)+'/1)... (legend '+str(legend+1)+'/'+str(poly_counter)+')... :', datetime.now()-runningtime_start)
            #for legend in range(0, poly_counter):
                #out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v4.png'
                #cv2.imwrite(out_file_path0, ans_category[legend])
            ans_category_temp = np.copy(ans_category)

            print('time check _v4:', datetime.now()-runningtime_start)
            running_time_v.append(datetime.now()-runningtime_start)




            conv_kernel_set = []
            conv_kernel_threshold0 = [1.0, 0.75, 0.75, 0.5, 0.5, 0.5, 0.5]#, 0.5, 0.5, 0.5]
            conv_kernel_threshold = []

            conv_kernel_0 = np.ones((3,3),dtype=np.uint8)
            conv_kernel_0[1,1] = 0
            conv_kernel_1 = np.ones((5,5),dtype=np.uint8)
            conv_kernel_1[2,2] = 0
            conv_kernel_2 = np.ones((7,7),dtype=np.uint8)
            conv_kernel_2[2:5,2:5] = 0
            conv_kernel_3 = np.ones((9,9),dtype=np.uint8)
            conv_kernel_3[3:6,3:6] = 0
            conv_kernel_4 = np.ones((11,11),dtype=np.uint8)
            conv_kernel_4[3:8,3:8] = 0
            conv_kernel_5 = np.ones((13,13),dtype=np.uint8)
            conv_kernel_5[4:9,4:9] = 0
            conv_kernel_6 = np.ones((15,15),dtype=np.uint8)
            conv_kernel_6[4:11,4:11] = 0

            conv_kernel_set.append(conv_kernel_0)
            conv_kernel_set.append(conv_kernel_1)
            conv_kernel_set.append(conv_kernel_2)
            conv_kernel_set.append(conv_kernel_3)
            conv_kernel_set.append(conv_kernel_4)
            conv_kernel_set.append(conv_kernel_5)
            conv_kernel_set.append(conv_kernel_6)

            for conv_set in range(0, len(conv_kernel_set)):
                conv_kernel_threshold.append(np.sum(conv_kernel_set[conv_set])*conv_kernel_threshold0[conv_set])



            boundingRange = 3
            masking0 = generate_mask(boundingRange)
            masking = np.copy(masking0)
            masking = masking.astype(float)

            for direction in range(0, 8):
                #print((masking[direction]==1.0).sum())
                region_sum = (masking[direction]==1.0).sum()
                for i, j in np.argwhere(masking[direction]==1.0):
                    masking[direction][i][j] = (1.0/region_sum)


            # keep record of updated region
            #updated_region = np.zeros(poly_counter)
            updated_region = []
            updated_for_relaxing = np.ones((ans_category[poly_counter].shape[0],ans_category[poly_counter].shape[1]),dtype=np.uint8)*255

            # fill ip white pixel (remove noisy black pixel)
            for iteration in range(0, 1):
                global_solution = np.copy(ans_category[poly_counter])
                global_solution_temp = np.copy(ans_category[poly_counter])
                global_solution_temp[global_solution_temp > 0] = 0
                global_solution_empty = 255 - global_solution_temp
                global_solution_empty = cv2.bitwise_and(global_solution_empty, img_bound)
                ans_category[poly_counter] = np.zeros((img_rb.shape[0],img_rb.shape[1]),dtype=np.uint8)

                #updated_region = []
                next_updated_region = []


                def extraction_step5(legend):
                    # fetch current result for this legend
                    this_current_result = np.copy(ans_category[legend])

                    if iteration == 0:
                        updated_for_relaxing = np.ones((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)*255
                        # remove moisy white pixels before buffer
                        kernel_before_blur = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        opening_before_blur = cv2.morphologyEx(this_current_result, cv2.MORPH_OPEN, kernel_before_blur, iterations=1)
                    else:
                        updated_for_relaxing = np.ones((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)*255
                        # remove moisy white pixels before buffer
                        kernel_before_blur = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        opening_before_blur = cv2.morphologyEx(this_current_result, cv2.MORPH_OPEN, kernel_before_blur, iterations=1)

                        print(np.sum(updated_for_relaxing))

                        if np.sum(updated_for_relaxing) == 0:
                            return legend, this_current_result, updated_for_relaxing
                        

                        masked_update = cv2.bitwise_and(rgb_rb, rgb_rb, mask=updated_for_relaxing)

                        color_dynamic_placeholder_0 = []
                        color_dynamic_placeholder_1 = []
                        for dimension in range(0, 3):
                            masked_update_np = np.copy(masked_update[:,:,dimension]).astype(float)
                            masked_update_np[masked_update_np==0] = np.nan

                            color_dynamic_placeholder_0.append(int(np.nanquantile(masked_update_np,.01)))
                            color_dynamic_placeholder_1.append(min(int(np.nanquantile(masked_update_np,.99)), 254))
                        print(color_dynamic_placeholder_0, color_dynamic_placeholder_1)


                    # smooth the image
                    if iteration == 0:
                        threshold_blur = 255*0.0
                    else:
                        threshold_blur = 255*0.0
                    blur_radius = 5.0
                    gaussian_buffer = ndimage.gaussian_filter(opening_before_blur, blur_radius)
                    gaussian_buffer[gaussian_buffer > threshold_blur] = 255
                    gaussian_buffer[gaussian_buffer <= threshold_blur] = 0

                    current_empty = 255 - this_current_result
                    current_relaxing = cv2.bitwise_and(current_empty, gaussian_buffer)
                    #current_relaxing = cv2.bitwise_and(current_relaxing, updated_for_relaxing) # shall not bitwise_and here <= already considered from gaussian_buffer <= opening_before_blur

                    current_relaxing = cv2. bitwise_and(current_relaxing, global_solution_empty)
                    #current_relaxing_arg = np.argwhere(current_relaxing == 255)

                    updated_for_relaxing = np.zeros((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)



                    this_current_result_scale = np.copy(this_current_result)
                    this_current_result_scale[this_current_result_scale > 0] = 1
                    this_current_result_scale_v0 = np.copy(this_current_result_scale)
                    if iteration > 0:
                        rgb_dynamic_lower_box = np.array([color_dynamic_placeholder_0[0], color_dynamic_placeholder_0[1], color_dynamic_placeholder_0[2]])
                        rgb_dynamic_upper_box = np.array([color_dynamic_placeholder_1[0], color_dynamic_placeholder_1[1], color_dynamic_placeholder_1[2]])
                        print(rgb_dynamic_lower_box, rgb_dynamic_upper_box)

                        # create a mask to only preserve current legend color in the basemap
                        rgb_masked_dynamic = cv2.inRange(rgb_rb, rgb_dynamic_lower_box, rgb_dynamic_upper_box)
                        rgb_masked_dynamic[rgb_masked_dynamic > 0] = 1
                        rgb_masked_dynamic = rgb_masked_dynamic.astype('uint8')

                        #masking_targeted_color = cv2.bitwise_and(current_relaxing, rgb_masked_dynamic)
                        #masking_black_and_gray = cv2.bitwise_and(current_relaxing, img_crop_black_and_gray)
                        #masking_self = cv2.bitwise_and(current_relaxing, this_current_result)

                        conv_masking_targeted = np.zeros((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)
                        for conv_set in range(0, len(conv_kernel_set)):
                            conv_mask_self = cv2.filter2D(src=this_current_result_scale, ddepth=-1, kernel=conv_kernel_set[conv_set])
                            conv_mask_relax = cv2.filter2D(src=rgb_masked_dynamic, ddepth=-1, kernel=conv_kernel_set[conv_set])
                            conv_mask_bg = cv2.filter2D(src=img_crop_black_and_gray, ddepth=-1, kernel=conv_kernel_set[conv_set])

                            conv_out_placeholder = np.copy(this_current_result_scale)
                            conv_out_placeholder[np.logical_and(conv_mask_self>=conv_kernel_threshold[conv_set], conv_mask_relax>=conv_kernel_threshold[conv_set], this_current_result_scale_v0==0)] = 1
                            conv_out_placeholder[conv_out_placeholder > 0] = 255
                            conv_masking_targeted = cv2.bitwise_or(conv_masking_targeted, conv_out_placeholder)


                        masking_targeted_conv = cv2.bitwise_and(current_relaxing, conv_masking_targeted)


                        updated_from_filtering = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                        updated_record = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                        updated_record[0:this_current_result_scale.shape[0], 0:this_current_result_scale.shape[1]] = 3
                        tabo_list = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)


                        for this_direction in range(0, 8+2):
                            direction = this_direction
                            if this_direction >= 8:
                                direction = this_direction - 8

                            dir_filtered = cv2.filter2D(src=this_current_result_scale, ddepth=-1, kernel=masking[direction]) # img_bw_for_filter
                            dir_filtered[dir_filtered <= 255*0.5] = 0
                            dir_filtered[dir_filtered > 255*0.5] = 255 / 8.0

                            #directional_filter.append(dir_filtered)
                            if this_direction < 8:
                                updated_from_filtering = cv2.add(updated_from_filtering, dir_filtered)

                            dir_record = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                            dir_record[dir_filtered > 0.0] = 1
                            updated_record = cv2.add(updated_record, dir_record)
                            updated_record = cv2.subtract(updated_record, np.ones((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8))
                            #print('unique value(s):', np.unique(updated_record))

                            if this_direction >= 3:
                                updated_tabo = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                                updated_tabo[updated_record <= 0] = 1
                                tabo_list = cv2.bitwise_or(tabo_list, updated_tabo)
                        
                        tabo_list[tabo_list > 0] = 255
                        tabo_list = tabo_list - 255

                        updated_with_checking = np.copy(updated_from_filtering)
                        updated_with_checking[updated_from_filtering < 255*3/8] = 0
                        updated_with_checking[updated_from_filtering >= 255*3/8] = 255
                        updated_with_checking[updated_from_filtering > 255*5/8] = 0
                        updated_with_checking = cv2.bitwise_and(updated_with_checking, tabo_list)

                        updated_from_filtering[updated_from_filtering <= 255*5/8] = 0
                        updated_from_filtering[updated_from_filtering > 255*5/8] = 255


                        this_next_result = cv2.bitwise_or(this_current_result, masking_targeted_conv)
                        this_next_result = cv2.bitwise_or(this_next_result, updated_from_filtering)
                    if iteration == 0:
                        conv_masking_targeted = np.zeros((this_current_result.shape[0],this_current_result.shape[1]),dtype=np.uint8)
                        for conv_set in range(0, len(conv_kernel_set)):
                            conv_mask = cv2.filter2D(src=this_current_result_scale, ddepth=-1, kernel=conv_kernel_set[conv_set])
                            conv_out_placeholder = np.copy(this_current_result_scale)
                            conv_out_placeholder[np.logical_and(conv_mask>=conv_kernel_threshold[conv_set], this_current_result_scale_v0==0)] = 1
                            conv_out_placeholder[conv_out_placeholder > 0] = 255
                            conv_masking_targeted = cv2.bitwise_or(conv_masking_targeted, conv_out_placeholder)

                        masking_targeted_conv = cv2.bitwise_and(current_relaxing, conv_masking_targeted)


                        updated_from_filtering = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                        updated_record = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                        updated_record[0:this_current_result_scale.shape[0], 0:this_current_result_scale.shape[1]] = 3
                        tabo_list = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)


                        for this_direction in range(0, 8+2):
                            direction = this_direction
                            if this_direction >= 8:
                                direction = this_direction - 8

                            dir_filtered = cv2.filter2D(src=this_current_result_scale, ddepth=-1, kernel=masking[direction]) # img_bw_for_filter
                            #print(np.unique(dir_filtered))
                            dir_filtered[dir_filtered <= 255*0.5] = 0
                            dir_filtered[dir_filtered > 255*0.5] = 255 / 8.0

                            #directional_filter.append(dir_filtered)
                            if this_direction < 8:
                                updated_from_filtering = cv2.add(updated_from_filtering, dir_filtered)

                            dir_record = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                            dir_record[dir_filtered > 0.0] = 1
                            updated_record = cv2.add(updated_record, dir_record)
                            updated_record = cv2.subtract(updated_record, np.ones((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8))
                            #print('unique value(s):', np.unique(updated_record))

                            if this_direction >= 3:
                                updated_tabo = np.zeros((this_current_result_scale.shape[0],this_current_result_scale.shape[1]),dtype=np.uint8)
                                updated_tabo[updated_record <= 0] = 1
                                tabo_list = cv2.bitwise_or(tabo_list, updated_tabo)
                        
                        tabo_list[tabo_list > 0] = 255
                        tabo_list = tabo_list - 255

                        updated_with_checking = np.copy(updated_from_filtering)
                        updated_with_checking[updated_from_filtering < 255*3/8] = 0
                        updated_with_checking[updated_from_filtering >= 255*3/8] = 255
                        updated_with_checking[updated_from_filtering > 255*5/8] = 0
                        updated_with_checking = cv2.bitwise_and(updated_with_checking, tabo_list)

                        updated_from_filtering[updated_from_filtering <= 255*5/8] = 0
                        updated_from_filtering[updated_from_filtering > 255*5/8] = 255


                        this_next_result = cv2.bitwise_or(this_current_result, masking_targeted_conv)
                        this_next_result = cv2.bitwise_or(this_next_result, updated_from_filtering)


                    updated_for_relaxing = cv2.subtract(this_next_result, this_current_result)
                    
                    if print_intermediate_image == True:
                        out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v5.png'
                        cv2.imwrite(out_file_path0, this_next_result)

                    return legend, this_next_result, updated_for_relaxing


                # multiprocessing_step5
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(extraction_step5, (this_poly,)) for this_poly in range(range_min, range_max)]

                    for this_poly in multiprocessing_results:
                        legend, this_next_result, updated_for_relaxing = this_poly.get()
                        # add masked result into private ans_category
                        ans_category[legend] = np.copy(this_next_result)
                        # add mophological result into global ans_category
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                        img_masked_morphology = cv2.morphologyEx(this_next_result, cv2.MORPH_OPEN, kernel, iterations=1)
                        img_masked_morphology[img_masked_morphology > 0] = legend+1
                        ans_category[poly_counter] = cv2.add(ans_category[poly_counter], img_masked_morphology)

                        next_updated_region.append(np.copy(updated_for_relaxing))





                updated_region = np.array(np.copy(next_updated_region))
                print('processing _v4 >>> _v5 (iteration '+str(iteration+1)+'/1)... (legend '+str(legend+1)+'/'+str(poly_counter)+')... :', datetime.now()-runningtime_start)
            #ans_category_updated = []
            #for legend in range(0, poly_counter):
                #out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v5.png'
                #cv2.imwrite(out_file_path0, ans_category[legend])

                #ans_category_updated_placeholder = cv2.subtract(ans_category[legend], ans_category_temp[legend])
                #ans_category_updated.append(ans_category_updated_placeholder)

            #ans_category_temp = np.copy(ans_category)
            print('time check _v5:', datetime.now()-runningtime_start)
            running_time_v.append(datetime.now()-runningtime_start)
            
            
            
            
            if poly_counter >= 5 and poly_counter <= 150:
                print('proceed to text detection...')

                img_backgroun_v0 = np.copy(img_rb)

                lower_black_text = np.array([0,0,0])
                upper_black_text = np.array([70,70,70])
                mask_box_text0 = cv2.inRange(img_backgroun_v0, lower_black_text, upper_black_text)
                res_box_text1 = cv2.bitwise_and(img_bound, img_bound, mask=mask_box_text0)
                threshold_text = cv2.medianBlur(res_box_text1,3)

                global_hsv_space = np.zeros((3, 400), dtype='uint8')
                local_hsv_space = np.zeros((poly_counter, 3, 400), dtype='uint8')
                #hsv_color_space = np.zeros((poly_counter, 2, 3), dtype='uint8')
                hsv_color_space = []

                for legend in range(range_min, range_max): ###
                    color_space_holder = []
                    color_space_holder.append(color_space[legend][0])
                    color_space_holder.append(color_space[legend][1])

                    this_hsv_color_space = np.copy(color_space_holder)
                    #hsv_color_space[legend] = np.copy(color_space_holder)
                    hsv_color_space.append(color_space_holder)
                    #print(legend_name[legend], color_space_holder, hsv_color_space[legend][1], this_hsv_color_space)

                    global_hsv_space[0][max(this_hsv_color_space[0][0]-1, 0): 1+this_hsv_color_space[1][0]+1] += 1 # h space
                    global_hsv_space[1][max(this_hsv_color_space[0][1]-15, 0): 1+this_hsv_color_space[1][1]+15] += 1 # s space
                    global_hsv_space[2][max(this_hsv_color_space[0][2]-15, 0): 1+this_hsv_color_space[1][2]+15] += 1 # v space
                    local_hsv_space[legend][0][max(this_hsv_color_space[0][0]-1, 0): 1+this_hsv_color_space[1][0]+1] = 1 # h space
                    local_hsv_space[legend][1][max(this_hsv_color_space[0][1]-15, 0): 1+this_hsv_color_space[1][1]+15] = 1 # s space
                    local_hsv_space[legend][2][max(this_hsv_color_space[0][2]-15, 0): 1+this_hsv_color_space[1][2]+15] = 1 # v space

                #print('legend loaded...')

                print('time check _text_v0:', datetime.now()-runningtime_start)
                running_time_v.append(datetime.now()-runningtime_start)




                def pre_v6_update(legend):
                    ### v6
                    # remove noisy white pixel
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    opening = cv2.morphologyEx(ans_category[legend], cv2.MORPH_OPEN, kernel, iterations=1)
                    return_image =cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY)[1]

                    if np.unique(ans_category[legend]).shape[0] != 2:
                        print('extract nothing...')
                        return_image = np.copy(img_bound)

                    return legend, return_image

                # multiprocessing
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(pre_v6_update, (this_poly,)) for this_poly in range(range_min, range_max)]

                    for this_poly in multiprocessing_results:
                        legend, return_image = this_poly.get()
                        ans_category[legend] = np.copy(return_image)
                print('v6 updated...')



                comparison_needed = []
                comparison_target = np.empty(poly_counter, dtype=object)


                #for legend in range(0, poly_counter):
                def specify_overlap_legend(legend):
                    has_similar_legend = False
                    overlapping_issue = False
                    similar_legend = []


                    if True:
                        for counter_legend in range(range_min, range_max):
                            if counter_legend == legend:
                                continue

                            # First, check huge simple overlapping
                            ## a loose restriction for color space
                            if np.sum(local_hsv_space[counter_legend][0][max(hsv_color_space[legend][0][0]-10, 0): 1+hsv_color_space[legend][1][0]+10]) > 0:
                                if np.sum(local_hsv_space[counter_legend][1][max(hsv_color_space[legend][0][1]-25, 0): 1+hsv_color_space[legend][1][1]+25]) > 0:
                                    if np.sum(local_hsv_space[counter_legend][2][max(hsv_color_space[legend][0][2]-25, 0): 1+hsv_color_space[legend][1][2]+25]) > 0:
                                        if np.mean(ans_category[legend]) > 0 and np.mean(ans_category[counter_legend]) > 0:
                                            ans_overlap = cv2.bitwise_and(ans_category[legend], ans_category[counter_legend])

                                            if (np.mean(ans_overlap) / np.mean(ans_category[legend])) > 0.66 and (np.mean(ans_overlap) / np.mean(ans_category[counter_legend])) > 0.66:
                                                # if there are few overlaps in v6 extracted answer, than we don't need text detection
                                                #print('we need to compare them')
                                                #print('overlapping issue with large area: '+legend_name[legend]+' <-> '+legend_name[counter_legend])
                                                similar_legend.append(counter_legend)
                                                overlapping_issue = True
                                                continue


                            # Second, check color overlapping
                            ## no need to proceed if there is no overlap in h space for this legend
                            if np.mean(global_hsv_space[0][max(hsv_color_space[legend][0][0]-1, 0): 1+hsv_color_space[legend][1][0]+1]) > 1:

                                # only compare legends with the same first character
                                if legend_name[legend][0] not in legend_name[counter_legend][0]:
                                    continue

                                combined_hsv_space = local_hsv_space[legend] + local_hsv_space[counter_legend]

                                if np.max(combined_hsv_space[0]) > 1 and np.max(combined_hsv_space[1]) > 1 and np.max(combined_hsv_space[2]) > 1:
                                    # if there are overlaps in all hsv spaces, than we probably need to proceed to text detection

                                    ans_overlap = cv2.bitwise_and(ans_category[legend], ans_category[counter_legend])
                                    if np.mean(ans_category[legend]) > 0 and np.mean(ans_category[counter_legend]) > 0:
                                        if (np.mean(ans_overlap) / np.mean(ans_category[legend])) > 0.2 and (np.mean(ans_overlap) / np.mean(ans_category[counter_legend])) > 0.2:
                                            # if there are few overlaps in v6 extracted answer, than we don't need text detection
                                            #print('we need to compare them')
                                            #print('overlapping issue with similar color: '+legend_name[legend]+' <-> '+legend_name[counter_legend])
                                            similar_legend.append(counter_legend)
                                            has_similar_legend = True
                    return legend, similar_legend


                # multiprocessing
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(specify_overlap_legend, (this_poly,)) for this_poly in range(range_min, range_max)]

                    for this_poly in multiprocessing_results:
                        legend, similar_legend = this_poly.get()
                        comparison_target[legend] = np.copy(similar_legend)
                        if len(similar_legend) > 0:
                            comparison_needed.append(legend)
                print(comparison_target)
                print(comparison_needed)

                print('time check _text_v1:', datetime.now()-runningtime_start)
                running_time_v.append(datetime.now()-runningtime_start)




                def distance_to_depot(x1, y1, x2, y2):
                    return math.pow(math.pow(x1-x2, 2)+math.pow(y1-y2, 2), 0.5)


                #global_res_probability = np.zeros((poly_counter, threshold_text.shape[0], threshold_text.shape[1]), dtype='uint8')
                global_res_probability = np.empty(poly_counter, dtype=object)
                global_confidence = np.empty(poly_counter, dtype=object)

                def find_legend_in_map(legend):
                    # fetch current result for this legend
                    this_current_result = np.copy(ans_category[legend])


                    img_legend_v0 = cv2.imread(os.path.join('intermediate7(2)', map_name, map_name+'_'+legend_name[legend]+'_poly_legend.tif'))
                    img_legend_v1 = img_legend_v0[int(img_legend_v0.shape[0]/8):img_legend_v0.shape[0]-int(img_legend_v0.shape[0]/8), int(img_legend_v0.shape[1]/8):img_legend_v0.shape[1]-int(img_legend_v0.shape[1]/8)]

                    lower_black_text = np.array([0,0,0])
                    upper_black_text = np.array([70,70,70])
                    mask_box_legend = cv2.inRange(img_legend_v1, lower_black_text, upper_black_text)
                    mask_box_legend = cv2.medianBlur(mask_box_legend,3)
                    #plt.imshow(mask_box_legend)
                    #plt.show()

                    # threshold_text = processed map with text highlighted
                    res = cv2.matchTemplate(threshold_text, mask_box_legend, cv2.TM_CCOEFF_NORMED)

                    #global_res_probability[legend] = np.copy(res) ###

                    #threshold = 0.75
                    #loc = np.where(res >= threshold)
                    #loc_arg = np.argwhere(res >= threshold)
                    #print(loc_arg.shape)
                    
                    
                    return legend, res
                    



                # multiprocessing
                list_for_multiprocessing = []
                for list_id in range(0, len(comparison_needed)):
                    legend = comparison_needed[list_id]
                    list_for_multiprocessing.append(legend)
                #print(list_for_multiprocessing)
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(find_legend_in_map, (this_poly,)) for this_poly in list_for_multiprocessing]

                    for this_poly in multiprocessing_results:
                        '''
                        legend, res, confidence_placeholder = this_poly.get()
                        global_res_probability[legend] = np.copy(res)
                        global_confidence[legend] = np.copy(confidence_placeholder)
                        '''
                        
                        legend, res = this_poly.get()
                        global_res_probability[legend] = np.copy(res)
                        
                        threshold = 0.75
                        loc = np.where(res >= threshold)
                        loc_arg = np.argwhere(res >= threshold)
                        while len(loc[0]) == 0 and threshold >= 0:
                            threshold = threshold - 0.1
                            loc = np.where(res >= threshold)
                            loc_arg = np.argwhere(res >= threshold)
                        global_confidence[legend] = np.copy(loc_arg)
                        

                print('time check _text_v2:', datetime.now()-runningtime_start)
                running_time_v.append(datetime.now()-runningtime_start)






                def generate_cluster(input_image, blur_radius_initial, blur_radius_step):
                    # smooth the image (to remove small objects)
                    blur_radius = blur_radius_initial
                    threshold_blur = 0
                    imgf = ndimage.gaussian_filter(input_image, blur_radius)

                    # find connected components
                    labeled, nr_objects = ndimage.label(imgf > threshold_blur)

                    while nr_objects > 100:
                        # smooth the image (to remove small objects)
                        blur_radius = blur_radius + blur_radius_step
                        threshold_blur = 0
                        imgf = ndimage.gaussian_filter(input_image, blur_radius)

                        # find connected components
                        labeled, nr_objects = ndimage.label(imgf > threshold_blur)


                    current_nr_objects = nr_objects
                    for object_traverse in range(1, nr_objects):
                        # if this object is too large (span from large x-y), split into pieces based on grid
                        cluster_object_arg = np.argwhere(labeled == object_traverse)
                        #print(cluster_object_arg.min(0), cluster_object_arg.max(0))

                        (y_min, x_min), (y_max, x_max) = cluster_object_arg.min(0), cluster_object_arg.max(0) + 1 
                        if (y_max-y_min) > 3000 and (x_max-x_min) > 3000:
                            #print(y_min,y_max, x_min,x_max)
                            this_labeled_object = np.copy(labeled)
                            this_labeled_object[this_labeled_object > object_traverse] = 0
                            this_labeled_object[this_labeled_object < object_traverse] = 0
                            this_labeled_object[this_labeled_object == object_traverse] = current_nr_objects

                            # construct a 1000*1000 grid
                            min_w = math.floor(y_min/1000)
                            max_w = math.floor(y_max/1000)
                            min_h = math.floor(x_min/1000)
                            max_h = math.floor(x_max/1000)

                            #print(min_w, max_w, min_h, max_h)
                            this_grid = np.zeros((ans_category[legend].shape[0], ans_category[legend].shape[1]), dtype='uint8')
                            for grid_w in range(min_w, max_w+1):
                                for grid_h in range(min_h, max_h+1):
                                    this_adding = (grid_w-min_w)*(max_h+1-min_h) + grid_h + 1
                                    this_grid[grid_w*1000:(grid_w+1)*1000, grid_h*1000:(grid_h+1)*1000] = this_adding

                            #updated_labeled = np.copy(labeled)
                            grid_count = (max_w-min_w)*(max_h-min_h)
                            for local_grid in range(0, grid_count):
                                #print(current_nr_objects+local_grid)
                                labeled[np.logical_and(labeled==object_traverse, this_grid==local_grid)] = (current_nr_objects+local_grid)

                            current_nr_objects = current_nr_objects+grid_count
                    nr_objects = current_nr_objects

                    return labeled, nr_objects


                def update_based_on_text(legend, counter_legend):
                    img_ans_v0 = np.copy(ans_category[legend])
                    #save_region_temp = cv2.subtract(ans_category[legend], ans_category[counter_legend])
                    temp_competitor = 255 - ans_category[counter_legend]
                    save_region_temp = cv2.bitwise_and(ans_category[legend], temp_competitor)
                    img_ans_v0 = cv2.subtract(img_ans_v0, save_region_temp)

                    labeled, nr_objects = generate_cluster(img_ans_v0, 15.0, 5.0)

                    depot_checked_polygon = np.zeros((labeled.shape[0],labeled.shape[1]),dtype=np.uint8)

                    depot_got = 0
                    distance_threshold = 300

                    for object_traverse in range(1, nr_objects):
                        cluster_object = np.argwhere(labeled == object_traverse)
                        if cluster_object.shape[0] == 0:
                            continue

                        center_x = np.mean(cluster_object, axis=0)[0]
                        center_y = np.mean(cluster_object, axis=0)[1]

                        belong_to_other_group = False
                        belong_to_this_group = False

                        # The text located 'in' this polygon => include
                        in_polygon_arg = np.logical_and(labeled[0:global_res_probability[legend].shape[0], 0:global_res_probability[legend].shape[1]] == object_traverse, global_res_probability[legend] >= 0.5)
                        in_polygon_bool = (True in in_polygon_arg)
                        if in_polygon_bool == True:
                            belong_to_other_group = False
                            belong_to_this_group = True

                        # The text located 'nearby' this polygon => include (especially for small polygons, where legends are labeled outside)
                        if belong_to_other_group == False:
                            center_placeholder = np.array([[center_x, center_y]])
                            center_placeholder = np.repeat(center_placeholder, len(global_confidence[legend]), axis=0)
                            distances_2_depot = np.sqrt(np.sum((global_confidence[legend]-center_placeholder)**2,axis=1))
                            min_distance_to_self = np.min(distances_2_depot)
                            
                            if min_distance_to_self < distance_threshold:
                                belong_to_other_group = False
                                belong_to_this_group = True

                        # The counter-text located 'in' this polygon => exclude
                        if belong_to_other_group == False:
                            center_placeholder = np.array([[center_x, center_y]])
                            center_placeholder = np.repeat(center_placeholder, len(global_confidence[counter_legend]), axis=0)
                            distances_2_depot = np.sqrt(np.sum((global_confidence[counter_legend]-center_placeholder)**2,axis=1))
                            min_distance_to_counter = np.min(distances_2_depot)
                            
                            in_polygon_arg = np.logical_and(labeled[0:global_res_probability[counter_legend].shape[0], 0:global_res_probability[counter_legend].shape[1]] == object_traverse, global_res_probability[counter_legend] >= 0.75)
                            in_polygon_bool = (True in in_polygon_arg)
                            if in_polygon_bool == True:
                                belong_to_other_group = True
                                belong_to_this_group = False
                            

                        if belong_to_other_group == False and belong_to_this_group == False:
                            #if min_distance_to_self/confidence_to_self < (min_distance_to_counter/confidence_to_counter)*1.33:
                            if min_distance_to_self < (min_distance_to_counter)*2.0:
                                belong_to_this_group = True


                        if belong_to_this_group == True and belong_to_other_group == False :
                            # create a mask to only preserve current legend color in the basemap
                            depot_checked_polygon_0 = np.zeros((labeled.shape[0],labeled.shape[1]),dtype=np.uint8)
                            depot_checked_polygon_0[np.logical_and(labeled==object_traverse, ans_category[legend]>0)] = 255
                            depot_checked_polygon_0 = cv2.bitwise_and(depot_checked_polygon_0, ans_category[legend])
                            depot_checked_polygon = cv2.bitwise_or(depot_checked_polygon, depot_checked_polygon_0)
                            depot_got = depot_got+1


                    #print('updated against - '+legend_name[counter_legend])
                    #plt.imshow(save_region_temp)
                    #plt.show()
                    #plt.imshow(depot_checked_polygon)
                    #plt.show()
                    img_ans_v1 = cv2.bitwise_and(ans_category[legend], depot_checked_polygon)
                    img_ans_v1 = cv2.bitwise_or(img_ans_v1, save_region_temp)
                    #plt.imshow(img_ans_v1)
                    #plt.show()

                    return img_ans_v1




                def compare_against_competitor(legend):
                    updated_region = np.copy(ans_category[legend])

                    for counter_list_id in range(0, len(comparison_target[legend])):
                        counter_legend = comparison_target[legend][counter_list_id]
                        #print(legend_name[legend]+' <-> '+legend_name[counter_legend])
                        img_ans_v1 = update_based_on_text(legend, counter_legend)
                        ban_region = cv2.subtract(ans_category[legend], img_ans_v1)
                        updated_region = cv2.subtract(updated_region, ban_region)
                    
                    
                    # remove noisy white pixel
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                    opening = cv2.morphologyEx(updated_region, cv2.MORPH_OPEN, kernel, iterations=1)
                    updated_region=cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY)[1]
                    
                    if poly_counter <= 40:
                        if np.unique(updated_region).shape[0] != 2 or (np.sum(updated_region)/np.unique(updated_region)[1]) / (np.sum(ans_category[legend])/np.unique(ans_category[legend])[1]) < 0.0005:
                            print(legend_name[legend]+' rollback...')
                            updated_region = np.copy(ans_category[legend])
                    else:
                        if np.unique(updated_region).shape[0] != 2 or (np.sum(updated_region)/np.unique(updated_region)[1]) / (np.sum(ans_category[legend])/np.unique(ans_category[legend])[1]) < 0.0001:
                            print(legend_name[legend]+' rollback...')
                            updated_region = np.copy(ans_category[legend])


                    if print_intermediate_image == True:
                        out_file_path000=os.path.join('intermediate7(2)', map_name, map_name+'_'+legend_name[legend]+'_poly_v6.png')
                        cv2.imwrite(out_file_path000, updated_region)

                    return legend, updated_region


                # multiprocessing
                temp_ans_category = np.copy(ans_category)
                list_for_multiprocessing = []
                for list_id in range(0, len(comparison_needed)):
                    legend = comparison_needed[list_id]
                    list_for_multiprocessing.append(legend)
                print(list_for_multiprocessing)
                with multiprocessing.Pool(PROCESSES) as pool:
                    multiprocessing_results = [pool.apply_async(compare_against_competitor, (this_poly,)) for this_poly in list_for_multiprocessing]

                    for this_poly in multiprocessing_results:
                        legend, updated_region = this_poly.get()
                        temp_ans_category[legend] = np.copy(updated_region)

                    print('time check _text_v3:', datetime.now()-runningtime_start)
                    running_time_v.append(datetime.now()-runningtime_start)


                    print('text detection finished...')
                    ans_category = np.copy(temp_ans_category)



            def extraction_step6(legend):
                '''
                # remove noisy white pixel
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
                opening = cv2.morphologyEx(ans_category[legend], cv2.MORPH_OPEN, kernel, iterations=1)
                ans_category[legend]=cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY)[1]
                '''


                if np.unique(ans_category[legend]).shape[0] != 2:
                    print('extract nothing...')
                    ans_category[legend] = np.copy(img_bound)


                if True: # print_intermediate_image == True:
                    out_file_path0='intermediate7(2)/'+map_name+'/'+map_name+'_'+legend_name[legend]+'_poly_v7.png'
                    cv2.imwrite(out_file_path0, ans_category[legend])

                # convert the grayscale image to binary image
                pred_binary_raster = ans_category[legend].astype(float) / 255

                # print
                #print('predicted binary raster:')
                #print('shape:', pred_binary_raster.shape)
                #print('unique value(s):', np.unique(pred_binary_raster))
                #print(map_name, '/', legend_name[legend])
                print(legend_name[legend])

                # plot the raster and save it
                plt.imshow(pred_binary_raster)
                plt.show()


                # save the raster into a .tif file
                out_file_path='intermediate7(2)/'+map_name+'_'+legend_name[legend]+'_poly.tif' # output
                pred_binary_raster=pred_binary_raster.astype('uint16')
                cv2.imwrite(out_file_path, pred_binary_raster)

                # convert the image to a binary raster .tif
                raster = rasterio.open(file_path)
                transform = raster.transform
                # array     = raster.read(1)
                crs       = raster.crs 
                width     = raster.width 
                height    = raster.height 

                raster.close()

                raster = rasterio.open(out_file_path)   
                array  = raster.read(1)
                raster.close()
                with rasterio.open(out_file_path, 'w', 
                                   driver    = 'GTIFF', 
                                   transform = transform, 
                                   dtype     = rasterio.uint8, 
                                   count     = 1, 
                                   compress  = 'lzw', 
                                   crs       = crs, 
                                   width     = width, 
                                   height    = height) as dst:

                    dst.write(array, indexes=1)
                    dst.close()
                    
                return legend, pred_binary_raster


            # multiprocessing_step6
            with multiprocessing.Pool(PROCESSES) as pool:
                multiprocessing_results = [pool.apply_async(extraction_step6, (this_poly,)) for this_poly in range(range_min, range_max)]

                for this_poly in multiprocessing_results:
                    legend, pred_binary_raster = this_poly.get()

            print('time check _v6:', datetime.now()-runningtime_start)
            running_time_v.append(datetime.now()-runningtime_start)



        if os.path.isfile('intermediate7(2)/'+'running_time_record_v2.csv') == False:
            with open('intermediate7(2)/'+'running_time_record_v2.csv','w') as fd:
                fd.write('File,checkpoint_1,checkpoint_2,checkpoint_3,checkpoint_4,checkpoint_5,checkpoint_6,\n')
                fd.close()
        with open('intermediate7(2)/'+'running_time_record_v2.csv','a') as fd:
            fd.write(map_name+',')
            for rtc in range(0, len(running_time_v)):
                fd.write(str(running_time_v[rtc])+',')
            fd.write('\n')
            fd.close()

